# [Diabetic retinopathy detection](https://www.kaggle.com/c/diabetic-retinopathy-detection/)

## [An overview of the diabetic retinopathy](https://nei.nih.gov/health/diabetic/retinopathy)
Diabetic retinopathy is a complication of diabetes affects the retinas of the eyes. The retinas are the part of the eye that detects ligh and converts into nerve signals that are then conveyed to the visual cortex in the brain, via the optic nerve. It is the most common cause of vision loss among individuals with diabetes.

It's caused by damage to the blood vessels of the light-sensitive tissue at the back of the eye (retina), due to the leaking of fluid from the retinal blood vessels or due to a due to a hemorrhage (bleeding) from these blood vessels. At first, diabetic retinopathy may cause no symptoms or only mild vision problems.

## Creating a deep neural network that detects the presence of diabetic retinopathy:

The dataset is a large set of high-resolution retina images taken under a variety of imaging conditions. A left and right field is provided for every subject. Images are labeled with a subject id as well as either left or right (e.g. 1_left.jpeg is the left eye of patient id 1).

A clinician has rated the presence of diabetic retinopathy in each image on a scale of 0 to 4.

This staging has direct relevance to progression of the disease.

Using these images and corresponding labels a deep convolutional neural network can be trained to detect retinal images for features of diabetic retinopathy.

## An overview of the diabetic retinopathy staging:

**1) Mild nonproliferative retinopathy:** Small areas of balloon-like swelling in the retina’s tiny blood vessels, called microaneurysms, occur at this earliest stage of the disease. These microaneurysms may leak fluid into the retina.

**2) Moderate nonproliferative retinopathy:** As the disease progresses, blood vessels that nourish the retina may swell and distort. They may also lose their ability to transport blood. Both conditions cause characteristic changes to the appearance of the retina and may contribute to DME.

**3) Severe nonproliferative retinopathy:** Many more blood vessels are blocked, depriving blood supply to areas of the retina. These areas secrete growth factors that signal the retina to grow new blood vessels.

**4) Proliferative diabetic retinopathy (PDR):** At this advanced stage, growth factors secreted by the retina trigger the proliferation of new blood vessels, which grow along the inside surface of the retina and into the vitreous gel, the fluid that fills the eye. The new blood vessels are fragile, which makes them more likely to leak and bleed. Accompanying scar tissue can contract and cause retinal detachment—the pulling away of the retina from underlying tissue, like wallpaper peeling away from a wall. Retinal detachment can lead to permanent vision loss.

## About the dataset:

The images in the dataset come from different models and types of cameras, which can affect the visual appearance of left vs. right. Some images are shown as one would see the retina anatomically (macula on the left, optic nerve on the right for the right eye). Others are shown as one would see through a microscope condensing lens (i.e. inverted, as one sees in a typical live eye exam). There are generally two ways to tell if an image is inverted:

It is inverted if the macula (the small dark central area) is slightly higher than the midline through the optic nerve. If the macula is lower than the midline of the optic nerve, it's not inverted.
If there is a notch on the side of the image (square, triangle, or circle) then it's not inverted. If there is no notch, it's inverted.

Like any real-world data set, you will encounter noise in both the images and labels. Images may contain artifacts, be out of focus, underexposed, or overexposed. A major aim of this competition is to develop robust algorithms that can function in the presence of noise and variation.

## Data labels:
* 0 - No DR
* 1 - Mild
* 2 - Moderate
* 3 - Severe
* 4 - Proliferative DR

In [0]:
setup = True
fetch_raw_data = False
colab_mode = True
upload_data = False
download_data = True
verbose = False

dataset_id = 'kaggle_diabetic_retinopathy'

In [0]:
import os 
import sys
import subprocess
import gc

In [0]:
def execute_in_shell(command=None, 
                     verbose = False):
    """ 
        command -- keyword argument, takes a list as input
        verbsoe -- keyword argument, takes a boolean value as input
    
        This is a function that executes shell scripts from within python.
        
        Keyword argument 'command', should be a list of shell commands.
        Keyword argument 'verbose', should be a boolean value to set verbose level.
        
        Example usage: execute_in_shell(command = ['ls ./some/folder/',
                                                    ls ./some/folder/  -1 | wc -l'],
                                        verbose = True ) 
                                        
        This command returns dictionary with elements: Output and Error.
        
        Output records the console output,
        Error records the console error messages.
                                        
    """
    error = []
    output = []
    
    if isinstance(command, list):
        for i in range(len(command)):
            try:
                process = subprocess.Popen(command[i], shell=True, stdout=subprocess.PIPE)
                process.wait()
                out, err = process.communicate()
                error.append(err)
                output.append(out)
                if verbose:
                    print ('Success running shell command: {}'.format(command[i]))
            except Exception as e:
                print ('Failed running shell command: {}'.format(command[i]))
                if verbose:
                    print(type(e))
                    print(e.args)
                    print(e)
                
    else:
        print ('The argument command takes a list input ...')
    return {'Output': output, 'Error': error }

In [0]:
command = ['pip3 install -q kaggle PyDrive scikit-optimize >/dev/null 2>&1',
           'mkdir ~/.kaggle/',
           'mkdir ./{}/'.format(dataset_id)]

In [5]:
if setup and colab_mode:
  execute_in_shell(command = command, 
                   verbose = True)

Success running shell command: pip3 install -q kaggle PyDrive scikit-optimize >/dev/null 2>&1
Success running shell command: mkdir ~/.kaggle/
Success running shell command: mkdir ./kaggle_diabetic_retinopathy/


In [0]:
if colab_mode:
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    from googleapiclient.http import MediaIoBaseDownload
    
import io
import glob
import fnmatch
import random

from multiprocessing import Process

import os, sys, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from imgaug import augmenters as iaa
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [7]:
import argparse
import os
import random
import time
import sys
import glob
try:
    import h5py
except:
    print ('Package h5py needed for saving model weights ...')
    sys.exit(1)
import json
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
try:
    import tensorflow
    import keras
except:
    print ('This code uses tensorflow deep-learning framework and keras api ...')
    print ('Install tensorflow and keras to train the classifier ...')
    sys.exit(1)
import PIL
from collections import defaultdict
from keras.applications.inception_v3 import InceptionV3,    \
                                            preprocess_input as preprocess_input_inceptionv3
from keras.applications.inception_resnet_v2 import InceptionResNetV2,    \
                                            preprocess_input as preprocess_input_inceptionv4
from keras.models import Model,                             \
                         model_from_json,                    \
                         load_model
from keras.layers import Dense,                             \
                         GlobalAveragePooling2D,            \
                         Dropout,                           \
                         BatchNormalization
from keras.layers.merge import concatenate
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.optimizers import SGD,                           \
                             RMSprop,                       \
                             Adagrad,                       \
                             Adadelta,                      \
                             Adam,                          \
                             Adamax,                        \
                             Nadam
from keras.callbacks import EarlyStopping,   \
                            ModelCheckpoint, \
                            ReduceLROnPlateau
                            
from multiprocessing import Process

Using TensorFlow backend.


## Authenticate notebook session to access Kaggle

The authentication of the notebook session in CoLab can be done via the Kaggle API. Download the kaggle.json file to your computer, from [Kaggle account settings](https://www.kaggle.com/{USERNAME}/account) under API, using the **"Create New API Token"** button. 

Upload that kaggle.json file to this session by executing the cell below.

In [0]:
if setup and fetch_raw_data and colab_mode:
  from google.colab import files
  uploaded = files.upload()

## Download the data

Using the Kaggle authentication token, the retinal imaging data will be downloaded, via the [Kaggle API](https://github.com/Kaggle/kaggle-api).

`kaggle competitions download -c diabetic-retinopathy-detection`

Before downloading the data, ensure that you have accepted the [Kaggle platform rules for the diabetic retinopathy detection competition](https://www.kaggle.com/c/diabetic-retinopathy-detection/rules).

In [0]:
command = ['mkdir ~/.kaggle/',
           'mv ./kaggle.json /root/.kaggle/',
           'chmod 600 ~/.kaggle/kaggle.json',
           'kaggle competitions download -c diabetic-retinopathy-detection']

In [0]:
if fetch_raw_data:
  execute_in_shell(command = command, verbose = True)

In [11]:
! ls 

kaggle_diabetic_retinopathy  sample_data


In [0]:
command = ['mv ./train.zip.001 ./train_{}.zip.001'.format(dataset_id),
           'mv ./train.zip.002 ./train_{}.zip.002'.format(dataset_id),
           'mv ./train.zip.003 ./train_{}.zip.003'.format(dataset_id),
           'mv ./train.zip.004 ./train_{}.zip.004'.format(dataset_id),
           'mv ./train.zip.005 ./train_{}.zip.005'.format(dataset_id),
           'mv ./trainLabels.csv.zip ./trainLabels_{}.csv.zip'.format(dataset_id),
           'mv ./test.zip.001 ./test_{}.zip.001'.format(dataset_id),
           'mv ./test.zip.002 ./test_{}.zip.002'.format(dataset_id),
           'mv ./test.zip.003 ./test_{}.zip.003'.format(dataset_id),
           'mv ./test.zip.004 ./test_{}.zip.004'.format(dataset_id),
           'mv ./test.zip.005 ./test_{}.zip.005'.format(dataset_id),
           'mv ./test.zip.006 ./test_{}.zip.006'.format(dataset_id),
           'mv ./test.zip.007 ./test_{}.zip.007'.format(dataset_id),
           'mv ./sample_submission.csv.zip ./sample_submission_{}'.format(dataset_id)]

In [0]:
if fetch_raw_data:
  execute_in_shell(command = command, verbose = True)

In [0]:
def cloud_authenticate():
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  print ("Sucessfully authenticated to access Google Drive ...")
  return drive

In [15]:
if colab_mode:
    drive = cloud_authenticate()

Sucessfully authenticated to access Google Drive ...


In [0]:
def googledrive_fetch(file_name = None, 
                fetch=True, 
                fetch_by_id = False,
                latest = True,
                file_id = None,
                multi_file = False):
  
  """
    A function that fetches files from Google Drive.
    
    The function takes five keyword arguments:
      file_name -- Passes the file name string
      fetch -- Specify if a file name should be downloaded
      fetch_by_id -- Specify a file to be downloaded by file id
      multi_file -- Download all the files with the same file name from Google Drive
  """
  
  query = 'title='+"'"+file_name+"'"
  try:
    file_list=drive.ListFile({'q': "{}".format(query)}).GetList()
  except:
    return ("Error finding file with {}".format(query))
  
  if len(file_list) >1:
    print ("A total of {} files with the same file name found ...".format(len(file_list)))
    for f in file_list:
      title = f['title']
      id = f.metadata.get('id')
      print ("Found: {} file, with file id: {}".format(title, id))
    
    if multi_file:
      print ("Downloading {} files with file name {}".format(len(file_list), title))
      print ("Staring download ...")
    elif latest:
      print ("Downloading the most recent {} file ...".format(title))
    elif file_id == None:
      print ("Set keyword argument fetch_by_id = True and specify id using keyword argument file_id = 'id' to download a specific file ...")
      print ("--OR--")
      print ("Set keyword argument multi_file = True to automatically download all the files ...")
      return None
    else:
      print ("Starting download ...")
    
  n = 0
  
  if latest:
    try:
      title = file_list[0]['title']
    except:
      return ("Error finding file with {}".format(query))
    latest_file_id = file_list[0].metadata.get('id')
    print ("Found most recent version of: {} file with file id: {} ...".format(title, latest_file_id))    
  
  for f in file_list:
      if fetch and multi_file and n>0:
        save_path = os.path.join('./'+str(n)+'_'+file_name)
      else:
        save_path = os.path.join('./'+file_name)     
      
      title = f['title']
      
      if fetch_by_id and file_id !=None:
        id = file_id
      elif latest:
        id = latest_file_id
      elif fetch_by_id and file_id == None:
        print ('Please specify the file id for downloading using the file_id argument ...')
      else:
        id = f.metadata.get('id')
      
      print ("Downloading {} file, with file id: {} ...".format(title, id))
      
      if fetch or fetch_by_id or latest:
        local_file = io.FileIO(save_path, mode='wb')
        try:
          request = drive.auth.service.files().get_media(fileId=id)
          downloader = MediaIoBaseDownload(local_file, request, chunksize=2048*102400)

          done = False

          while done is False:
              status, done = downloader.next_chunk()
        except:
          return 'Downloading failed ...'
        
        local_file.close()
        print ("Successfully downloaded the file: {} to: {} ...".format(file_name, save_path))
      
      if fetch_by_id and file_id !=None:
        return None
      elif latest:
        return None
      elif n >= 0:
        print ("Downloaded {} of {} files ...".format(n+1, len(file_list)))
      else:
        print ("Download failed ...")
      
      n +=1
  
  return None

In [0]:
def googledrive_save(file_name = None, 
               file_dir = None, 
               upload = False,
               prefix = None):
  if upload == True and file_name != None and file_dir !=None:
    try:
      if prefix != None:
        file = drive.CreateFile({'title': str(prefix) + str(file_name) })
      else:
        file = drive.CreateFile({'title': str(file_name) })
      file.SetContentFile(os.path.join(file_dir + str(file_name)))
      file.Upload()
      print (str(file_name) + " successfully uploaded to Google drive ...")
    except:
      print ("Failed to save :" + str(file_name) + " to Google drive ...")

In [0]:
file_dir = './'
file_name = ['test_{}.zip.001'.format(dataset_id),
             'test_{}.zip.002'.format(dataset_id),
             'test_{}.zip.003'.format(dataset_id),
             'test_{}.zip.004'.format(dataset_id),
             'test_{}.zip.005'.format(dataset_id),
             'test_{}.zip.006'.format(dataset_id),
             'test_{}.zip.007'.format(dataset_id),
             'train_{}.zip.001'.format(dataset_id),
             'train_{}.zip.002'.format(dataset_id),
             'train_{}.zip.003'.format(dataset_id),
             'train_{}.zip.004'.format(dataset_id),
             'train_{}.zip.005'.format(dataset_id),
             'trainLabels_{}.csv.zip'.format(dataset_id)]

## Upload the dataset to Google drive

In [0]:
if upload_data and colab_mode:
  for f in file_name:
    googledrive_save(file_name = f,
                     file_dir = file_dir,
                     upload = True)

In [20]:
if download_data and colab_mode:
  for f in file_name:
    googledrive_fetch(file_name = f, 
                      fetch=True, 
                      latest = True)

Found most recent version of: test_kaggle_diabetic_retinopathy.zip.001 file with file id: 1ixiYl3gTjMJYxYYOkH__22toa0bFOF4x ...
Successfully downloaded the file: test_kaggle_diabetic_retinopathy.zip.001 to: ./test_kaggle_diabetic_retinopathy.zip.001 ...
Found most recent version of: test_kaggle_diabetic_retinopathy.zip.002 file with file id: 110H1wy_8FqyyRGZOi-_a8PuUqgCKlTc1 ...
Successfully downloaded the file: test_kaggle_diabetic_retinopathy.zip.002 to: ./test_kaggle_diabetic_retinopathy.zip.002 ...
Found most recent version of: test_kaggle_diabetic_retinopathy.zip.003 file with file id: 1KiDf4ypex6A8VMkGWx-Br1jamOSFVB52 ...
Successfully downloaded the file: test_kaggle_diabetic_retinopathy.zip.003 to: ./test_kaggle_diabetic_retinopathy.zip.003 ...
Found most recent version of: test_kaggle_diabetic_retinopathy.zip.004 file with file id: 1JJs7HYeQOhRJtrFSmCvtY9ZlkF3kf140 ...
Successfully downloaded the file: test_kaggle_diabetic_retinopathy.zip.004 to: ./test_kaggle_diabetic_retinopa

In [0]:
command = ['mkdir ./{}/'.format(dataset_id),
           'mkdir ./{}/train/'.format(dataset_id),
           'sudo apt-get install p7zip-full',
           '7z e ./train_{}.zip.001 -o./{}/train/ -r'.format(dataset_id,
                                                             dataset_id),
           '7z e ./trainLabels_{}.csv.zip -o./{}/'.format(dataset_id,
                                                        dataset_id),
           'rm ./train_{}.zip.001'.format(dataset_id),
           'mkdir ./{}/train/0/'.format(dataset_id),
           'mkdir ./{}/train/1/'.format(dataset_id),
           'mkdir ./{}/train/2/'.format(dataset_id),
           'mkdir ./{}/train/3/'.format(dataset_id),
           'mkdir ./{}/train/4/'.format(dataset_id)]

In [22]:
if setup:
  execute_in_shell(command = command, 
                   verbose = True)

Success running shell command: mkdir ./kaggle_diabetic_retinopathy/
Success running shell command: mkdir ./kaggle_diabetic_retinopathy/train/
Success running shell command: sudo apt-get install p7zip-full
Success running shell command: 7z e ./train_kaggle_diabetic_retinopathy.zip.001 -o./kaggle_diabetic_retinopathy/train/ -r
Success running shell command: 7z e ./trainLabels_kaggle_diabetic_retinopathy.csv.zip -o./kaggle_diabetic_retinopathy/
Success running shell command: rm ./train_kaggle_diabetic_retinopathy.zip.001
Success running shell command: mkdir ./kaggle_diabetic_retinopathy/train/0/
Success running shell command: mkdir ./kaggle_diabetic_retinopathy/train/1/
Success running shell command: mkdir ./kaggle_diabetic_retinopathy/train/2/
Success running shell command: mkdir ./kaggle_diabetic_retinopathy/train/3/
Success running shell command: mkdir ./kaggle_diabetic_retinopathy/train/4/


In [62]:
! ls ./kaggle_diabetic_retinopathy/validation/4/  | wc -l

100


## Label the data

In [0]:
import pandas as pd
import os

CSV_FILE = os.path.join('./{}/trainLabels.csv'.format(dataset_id))

In [0]:
labels = pd.read_csv(CSV_FILE)
file_names = labels.image
file_labels = labels.level

In [0]:
for i in range(len(file_names)):
  file_path = './{}/train/{}.jpeg'.format(dataset_id,
                                     file_names[i])
  labelled_dir = './{}/train/{}/'.format(dataset_id,
                                         file_labels[i])
  labelled_path = '{}/{}.jpeg'.format(labelled_dir,
                                 file_names[i])
  if os.path.exists(file_path) and os.path.exists(labelled_dir):
    if verbose:
      print ("File found: {}".format(file_path))
    os.rename(file_path, labelled_path)
    if verbose:
      print ("File moved from: {} ;to: {} ...".format(file_path,
                                                      labelled_path))

In [0]:
command = ['mkdir ./{}/validation/'.format(dataset_id),
           'mkdir ./{}/validation/0/'.format(dataset_id),
           'mkdir ./{}/validation/1/'.format(dataset_id),
           'mkdir ./{}/validation/2/'.format(dataset_id),
           'mkdir ./{}/validation/3/'.format(dataset_id),
           'mkdir ./{}/validation/4/'.format(dataset_id),
           'cd ./{}/train/0/ ; shuf -n 100 -e * | xargs -i mv {} ../../../{}/validation/0/'.format(dataset_id,
                                                                                                          '{}',
                                                                                                          dataset_id),

           'cd ./{}/train/1/ ; shuf -n 100 -e * | xargs -i mv {} ../../../{}/validation/1/'.format(dataset_id,
                                                                                                          '{}',
                                                                                                          dataset_id),
           'cd ./{}/train/2/ ; shuf -n 100 -e * | xargs -i mv {} ../../../{}/validation/2/'.format(dataset_id,
                                                                                                    '{}',
                                                                                                    dataset_id),
           'cd ./{}/train/3/ ; shuf -n 100 -e * | xargs -i mv {} ../../../{}/validation/3/'.format(dataset_id,
                                                                                                    '{}',
                                                                                                    dataset_id),
           'cd ./{}/train/4/ ; shuf -n 100 -e * | xargs -i mv {} ../../../{}/validation/4/'.format(dataset_id,
                                                                                                    '{}',
                                                                                                    dataset_id)]

In [61]:
if setup:
  execute_in_shell(command = command, verbose = True)

Success running shell command: mkdir ./kaggle_diabetic_retinopathy/validation/
Success running shell command: mkdir ./kaggle_diabetic_retinopathy/validation/0/
Success running shell command: mkdir ./kaggle_diabetic_retinopathy/validation/1/
Success running shell command: mkdir ./kaggle_diabetic_retinopathy/validation/2/
Success running shell command: mkdir ./kaggle_diabetic_retinopathy/validation/3/
Success running shell command: mkdir ./kaggle_diabetic_retinopathy/validation/4/
Success running shell command: cd ./kaggle_diabetic_retinopathy/train/0/ ; shuf -n 100 -e * | xargs -i mv {} ../../../kaggle_diabetic_retinopathy/validation/0/
Success running shell command: cd ./kaggle_diabetic_retinopathy/train/1/ ; shuf -n 100 -e * | xargs -i mv {} ../../../kaggle_diabetic_retinopathy/validation/1/
Success running shell command: cd ./kaggle_diabetic_retinopathy/train/2/ ; shuf -n 100 -e * | xargs -i mv {} ../../../kaggle_diabetic_retinopathy/validation/2/
Success running shell command: cd ./

In [0]:
def generate_timestamp():
    """ 
        A function to generate time-stamp information.
        Calling the function returns a string formatted current system time.
        Eg: 2018_10_10_10_10_10
    
        Example usage: generate_timestamp() 
    """    
    timestring = time.strftime("%Y_%m_%d-%H_%M_%S")
    print ("Time stamp generated: " + timestring)
    return timestring

In [31]:
timestr = generate_timestamp()

Time stamp generated: 2019_03_12-23_38_11


In [0]:
def is_valid_file(parser, arg):
    """
        A function that checks if a give file path contains a valid file or not.
        
        The function returns the full file path if there is a valid file persent.
        If there is no valid file present at a file path location, it returns a parser error message.
        
        Takes two positional arguments: parser and arg
        
        Example usage: 
            import argsparse
            
            a = argparse.ArgumentParser()
            a.add_argument("--file_path", 
                              help = "Check if a file exists in the specified file path ...", 
                              dest = "file_path", 
                              required=False,
                              type=lambda x: is_valid_file(a, x),
                              nargs=1)
            
            args = a.parse_args()
            
            args = get_user_options()
    """
    if not os.path.isfile(arg):
        try:
            parser.error("The file %s does not exist ..." % arg)
            return None
        except:
            if parser != None:
                print ("No valid argument parser found ...")
                print ("The file %s does not exist ..." % arg)
                return None
            else:
                print ("The file %s does not exist ..." % arg)
                return None
    else:
        return arg

In [0]:
def is_valid_dir(parser, arg):
    """
        This function checks if a directory exists or not.
        It can be used inside the argument parser.
        
        Example usage: 
            
            import argsparse
            
            a = argparse.ArgumentParser()
            a.add_argument("--dir_path", 
                              help = "Check if a file exists in the specified file path ...", 
                              dest = "file_path", 
                              required=False,
                              type=lambda x: is_valid_dir(a, x),
                              nargs=1)
            
            args = a.parse_args()
            
            args = get_user_options() 
    """
    if not os.path.isdir(arg):
        try:
            return parser.error("The folder %s does not exist ..." % arg)
        except:
            if parser != None:
                print ("No valid argument parser found")
                print ("The folder %s does not exist ..." % arg)
                return None
            else:
                print ("The folder %s does not exist ..." % arg)
                return None
    else:
        return arg

In [0]:
def string_to_bool(val):
    """
        A function that checks if an user argument is boolean or not.
        
        Example usage:
            
            
                import argsparse
            
                a = argparse.ArgumentParser()
                
                a.add_argument("--some_bool_arg", 
                   help = "Specify a boolean argument ...", 
                   dest = "some_bool_arg", 
                   required=False, 
                   default=[True], 
                   nargs=1, 
                   type = string_to_bool)
                
            args = a.parse_args()
            
            args = get_user_options()
            
    """
    if val.lower() in ('yes', 'true', 't', 'y', '1', 'yeah', 'yup'):
        return True
    elif val.lower() in ('no', 'false', 'f', 'n', '0', 'none', 'nope'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected ...')

In [0]:
def activation_val(val):
    activation_function_options = ('hard_sigmoid',
                                   'elu',
                                   'linear',
                                   'relu', 
                                   'selu', 
                                   'sigmoid',
                                   'softmax',
                                   'softplus',
                                   'sofsign',
                                   'tanh')
    if val.lower() in activation_function_options:
        return val
    else:
        raise argparse.ArgumentTypeError('Unexpected activation function. \
                                         \nExpected values are:  {} ...'.format(activation_function_options))

In [0]:
def loss_val(val):
    loss_function_options = ('mean_squared_error',
                             'mean_absolute_error',
                             'mean_absolute_percentage_error',
                             'mean_squared_logarithmic_error', 
                             'squared_hinge', 
                             'hinge',
                             'categorical_hinge',
                             'logcosh',
                             'categorical_crossentropy',
                             'sparse_categorical_crossentropy',
                             'binary_crossentropy',
                             'kullback_leibler_divergence',
                             'poisson',
                             'cosine_proximity')
    if val.lower() in loss_function_options:
        return val
    else:
        raise argparse.ArgumentTypeError('Unexpected loss function. \
                                         \nExpected values are:  {} ...'.format(loss_function_options))
        


In [0]:
def get_nb_files(directory):
  if not os.path.exists(directory):
    return 0
  cnt = 0
  for r, dirs, files in os.walk(directory):
    for dr in dirs:
      cnt += len(glob.glob(os.path.join(r, dr + "/*")))
  return cnt

In [0]:
def add_top_layer(args, enable_dropout, base_model, nb_classes):
  """
    This functions adds a fully connected convolutional neural network layer to a base model.
    
    The required input arguments for this function are: args, base_model and nb_classes.
        args: argument inputs the user arguments to be passed to the function,
        base_model: argument inputs the base model architecture to be added to the top layer,
        nb_classes: argument inputs the total number of classes for the output layer.    
  """
  try:
      dropout = float(args.dropout[0])
      weight_decay = float(args.decay[0])
  except:
      dropout = DEFAULT_DROPOUT
      print ('Invalid input for dropout ...')
      
  try:
      activation = str(args.activation[0]).lower()
      print ('Building model using activation function: ' + str(activation))
  except:
      activation = 'relu'
      print ('Invalid input for activation function ...')
      print ('Choice of activation functions: hard_sigmoid, elu, linear, relu, selu, sigmoid, softmax, softplus, sofsign, tanh ...')
      print ('Building model using default activation function: relu')
      
  bm = base_model.output
  
  x = Dropout(dropout,
              name='dropout_fc1')(bm,
                       training=enable_dropout)
  x = GlobalAveragePooling2D(name='gloablAveragePooling2D_fc1')(x)
  x = Dropout(dropout,
              name='dropout_fc2')(x,
                       training=enable_dropout)
  x = BatchNormalization(name='batchNormalization_fc1')(x)
  x = Dense(FC_SIZE, 
            activation=activation,
            kernel_regularizer=l2(weight_decay),
            name='dense_fc1')(x)
  x = Dropout(dropout,
              name='dropout_fc3')(x,
                       training=enable_dropout)
  
  x1 = Dense(FC_SIZE, 
             activation=activation,
             kernel_regularizer=l2(weight_decay),
             name="dense_fc2")(x)
  x1 = Dropout(dropout,
               name = 'dropout_fc4')(x1, 
                                  training=enable_dropout)
  x1 = BatchNormalization(name="batchNormalization_fc2")(x1)
  x1 = Dense(FC_SIZE, 
             activation=activation, 
             kernel_regularizer=l2(weight_decay),
             name="dense_fc3")(x1)
  x1 = Dropout(dropout,
               name = 'dropout_fc5')(x1, 
                                  training=enable_dropout)

  x2 = Dense(FC_SIZE, 
             activation=activation, 
             kernel_regularizer=l2(weight_decay),
             name="dense_fc4")(x)
  x2 = Dropout(dropout,
               name = 'dropout_fc6')(x2, 
                                  training=enable_dropout)
  x2 = BatchNormalization(name="batchNormalization_fc3")(x2)
  x2 = Dense(FC_SIZE, 
             activation=activation, 
             kernel_regularizer=l2(weight_decay),
             name="dense_fc5")(x2)
  x2 = Dropout(dropout,
               name = 'dropout_fc7')(x2, 
                                  training=enable_dropout)

  x12 = concatenate([x1, x2], name = 'mixed11')
  x12 = Dropout(dropout,
                name = 'dropout_fc8')(x12, 
                                   training=enable_dropout)
  x12 = Dense(FC_SIZE//16, 
              activation=activation, 
              kernel_regularizer=l2(weight_decay),
              name = 'dense_fc6')(x12)
  x12 = Dropout(dropout,
                name = 'dropout_fc9')(x12, 
                                   training=enable_dropout)
  x12 = BatchNormalization(name="batchNormalization_fc4")(x12)
  x12 = Dense(FC_SIZE//32, 
              activation=activation, 
              kernel_regularizer=l2(weight_decay),
              name = 'dense_fc7')(x12)
  x12 = Dropout(dropout,
                name = 'dropout_fc10')(x12, 
                                   training=enable_dropout)
  
  x3 = Dropout(dropout,
              name='dropout_fc11')(bm,
                       training=enable_dropout)
  x3 = GlobalAveragePooling2D( name = 'globalAveragePooling2D_fc2')(x3)
  x3 = Dense(FC_SIZE//2, 
             activation=activation, 
             kernel_regularizer=l2(weight_decay),
             name = 'dense_fc8')(x3)
  x3 = Dropout(dropout,
               name = 'dropout_fc12')(x3, 
                                  training=enable_dropout)
  x3 = BatchNormalization(name="batchNormalization_fc5")(x3)
  x3 = Dense(FC_SIZE//2, 
             activation=activation, 
             kernel_regularizer=l2(weight_decay),
             name = 'dense_fc9')(x3)
  x3 = Dropout(dropout,
               name = 'dropout_fc13')(x3, 
                                  training=enable_dropout)
  
  xout = concatenate([x12, x3], name ='mixed12')
  xout = Dense(FC_SIZE//32, 
               activation= activation, 
               kernel_regularizer=l2(weight_decay),
               name = 'dense_fc10')(xout)
  xout = Dropout(dropout,
                 name = 'dropout_fc14')(xout, 
                                     training=enable_dropout)
  
  predictions = Dense(nb_classes,           \
                      activation='sigmoid', \
                      kernel_regularizer=l2(weight_decay),
                      name='prediction')(xout) # Softmax output layer
  
  model = Model(inputs=base_model.input, 
                outputs=predictions)
  
  return model

In [0]:
def finetune_model(model, optimizer, loss, NB_FROZEN_LAYERS):
  """
      A function that freezes the bottom NB_LAYERS and retrain the remaining top layers.
      
      The required input arguments for this function are: model, optimizer and NB_FROZEN_LAYERS.
          model: inputs a model architecture with base layers to be frozen during training,
          optimizer: inputs a choice of optimizer value for compiling the model,
          loss: inputs a choice for loss function used for compiling the model,
          NB_FROZEN_LAYERS: inputs a number that selects the total number of base layers to be frozen during training.
      
  """
                     
  for layer in model.layers[:NB_FROZEN_LAYERS]:
     layer.trainable = False
  for layer in model.layers[NB_FROZEN_LAYERS:]:
     layer.trainable = True
  model.compile(optimizer=optimizer, 
                loss=loss, 
                metrics=['accuracy'])
  return model

In [0]:
def transferlearn_model(model, base_model, optimizer, loss):
  """
     Function that freezes the base layers to train just the top layer.
     
     This function takes three positional arguments:
         model: specifies the input model,
         base_model: specifies the base model architecture,
         optimizer: optimizer function for training the model,
         loss: loss function for compiling the model
     
     Example usage:
         transferlearn_model(model, base_model, optimizer)
  """
  for layer in base_model.layers:
    layer.trainable = False
  model.compile(optimizer=optimizer, 
                loss=loss, 
                metrics=['accuracy'])
  return model

In [0]:
def save_model(args, name, model):
    file_loc = args.output_dir[0]
    file_pointer = os.path.join(file_loc+"//trained_"+ timestr)
    model.save_weights(os.path.join(file_pointer + "_weights"+str(name)+".model"))
    
    model_json = model.to_json()                                                # Serialize model to JSON
    with open(os.path.join(file_pointer+"_config"+str(name)+".json"), "w") as json_file:
        json_file.write(model_json)
    print ("Saved the trained model weights to: " + 
           str(os.path.join(file_pointer + "_weights"+str(name)+".model")))
    print ("Saved the trained model configuration as a json file to: " + 
           str(os.path.join(file_pointer+"_config"+str(name)+".json")))

In [0]:
def generate_labels(args):
    file_loc = args.output_dir[0]
    file_pointer = os.path.join(file_loc+"//trained_labels")
    
    data_dir = args.train_dir[0]
    val_dir_ = args.val_dir[0]
    
    dt = defaultdict(list)
    dv = defaultdict(list)
    
    for root, subdirs, files in os.walk(data_dir):
        for filename in files:
            file_path = os.path.join(root, filename)
            assert file_path.startswith(data_dir)
            suffix = file_path[len(data_dir):]
            suffix = suffix.lstrip("/")
            label = suffix.split("/")[0]
            dt[label].append(file_path)
            
    for root, subdirs, files in os.walk(val_dir_):
        for filename in files:
            file_path = os.path.join(root, filename)
            assert file_path.startswith(val_dir_)
            suffix = file_path[len(val_dir_):]
            suffix = suffix.lstrip("/")
            label = suffix.split("/")[0]
            dv[label].append(file_path)

    labels = sorted(dt.keys())
    val_labels = sorted(dv.keys())
    
    if set(labels) == set (val_labels):
        print("\nTraining labels: " + str(labels))
        print("\nValidation labels: " + str(val_labels))
        with open(os.path.join(file_pointer+".json"), "w") as json_file:
            json.dump(labels, json_file)
    else:
      print("\nTraining labels: " + str(labels))
      print("\nValidation labels: " + str(val_labels))
      print ("Mismatched training and validation data labels ...")
      print ("Sub-folder names do not match between training and validation directories ...")
      sys.exit(1)

    return labels


In [0]:
def normalize(args, 
              labels, 
              move = False, 
              sub_sample = False):
    if args.normalize[0] and os.path.exists(args.root_dir[0]):      
        commands = ["rm -r {}/.tmp_train/".format(args.root_dir[0]),
                    "rm -r {}/.tmp_validation/".format(args.root_dir[0]),
                    "mkdir {}/.tmp_train/".format(args.root_dir[0]),
                    "mkdir {}/.tmp_validation/".format(args.root_dir[0])]
        execute_in_shell(command=commands,
                         verbose=verbose)
        del commands
        
        mk_train_folder = "mkdir -p {}/.tmp_train/".format(args.root_dir[0]) + "{}"
        mk_val_folder = "mkdir -p {}/.tmp_validation/".format(args.root_dir[0]) + "{}"
        
        train_class_sizes = []
        val_class_sizes = []
        
        for label in labels:
            train_class_sizes.append(len(glob.glob(args.train_dir[0] + "/{}/*".format(label))))
            val_class_sizes.append(len(glob.glob(args.val_dir[0] + "/{}/*".format(label))))
        
        train_size = min(train_class_sizes)
        val_size = min(val_class_sizes)
        
        if sub_sample and 0 <= args.train_sub_sample[0] <=1 and 0 <= args.val_sub_sample[0] <=1 :
            train_size = int(train_size * args.train_sub_sample[0])
            val_size = int(val_size * args.val_sub_sample[0])
        
        print ("Normalized training class size {}".format(train_size))
        print ("Normalized validation class size {}".format(val_size))
        
        for label in labels:
            commands = [mk_train_folder.format(label),
                        mk_val_folder.format(label)]
        
            execute_in_shell(command=commands,
                             verbose=verbose)
            del commands
        
        commands = []
        
        for label in labels:
            train_images = (glob.glob('{}/{}/*.*'.format(args.train_dir[0], label), recursive=True))
            val_images = (glob.glob('{}/{}/*.*'.format(args.val_dir[0], label), recursive=True))
            
            sys_rnd = random.SystemRandom()
            
            if move:
              cmd = 'mv'
            else:
              cmd = 'cp'
            
            for file in sys_rnd.sample(train_images, train_size):
                if os.path.exists(file):
                    commands.append('{} {} ./.tmp_train/{}/'.format(cmd, file, label))
            
            for file in sys_rnd.sample(val_images, val_size):
                if os.path.exists(file):
                    commands.append('{} {} ./.tmp_validation/{}/'.format(cmd, file, label))
                
            p = Process(target=execute_in_shell, args=([commands]))
            p.start()
            p.join()
        print ("\nData normalization pipeline completed successfully ...")
    else:
        print ("\nFailed to initiate data normalization pipeline ...")
        return False    
    return True

In [0]:
def generate_plot(args, name, model_train):
    gen_plot = args.plot[0]
    if gen_plot==True:
        plot_training(args, name, model_train)
    else:
        print ("\nNo training summary plots generated ...")
        print ("Set: --plot True for creating training summary plots")


In [0]:
def plot_training(args, name, history):
  output_loc = args.output_dir[0]
  
  output_file_acc = os.path.join(output_loc+
                                 "//training_plot_acc_" + 
                                 timestr+str(name)+".png")
  output_file_loss = os.path.join(output_loc+
                                  "//training_plot_loss_" + 
                                  timestr+str(name)+".png")
  fig_acc = plt.figure()
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  fig_acc.savefig(output_file_acc, dpi=fig_acc.dpi)
  print ("Successfully created the training accuracy plot: " 
         + str(output_file_acc))
  plt.close()

  fig_loss = plt.figure()
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  fig_loss.savefig(output_file_loss, dpi=fig_loss.dpi)
  print ("Successfully created the loss function plot: " 
         + str(output_file_loss))
  plt.close()

In [0]:
def select_optimizer(args):
  optimizer_val = args.optimizer_val[0]
  lr = args.learning_rate[0]
  decay = args.decay[0]
  epsilon = args.epsilon[0]
  rho = args.rho[0]
  beta_1 = args.beta_1[0]
  beta_2 = args.beta_2[0]
  
  if optimizer_val.lower() == 'sgd' :
    optimizer = SGD(lr=lr,       \
                    decay=decay, \
                    momentum=1,  \
                    nesterov=False)
    print ("Using SGD as the optimizer ...")
  elif optimizer_val.lower() == 'nsgd':
    optimizer = SGD(lr=lr,      \
                    decay=decay,\
                    momentum=1, \
                    nesterov=True)
    print ("Using SGD as the optimizer with Nesterov momentum ...")
  elif optimizer_val.lower() == 'rms' \
       or \
       optimizer_val.lower() == 'rmsprop':
    optimizer = RMSprop(lr=lr,          \
                        rho=rho,        \
                        epsilon=epsilon,\
                        decay=decay)
    print ("Using RMSProp as the optimizer ...")
  elif optimizer_val.lower() == 'ada' \
       or \
       optimizer_val.lower() == 'adagrad':
    optimizer = Adagrad(lr=lr,           \
                        epsilon=epsilon, \
                        decay=decay)
    print ("Using Adagrad as the optimizer ...")
  elif optimizer_val.lower() == 'adelta' \
       or \
       optimizer_val.lower() == 'adadelta':
    optimizer = Adadelta(lr=lr,           \
                         rho=rho,         \
                         epsilon=epsilon, \
                         decay=decay)
    print ("Using Adadelta as the optimizer ...")
  elif optimizer_val.lower() == 'adam':
    optimizer = Adam(lr=lr,           \
                     beta_1=beta_1,   \
                     beta_2=beta_2,    \
                     epsilon=epsilon, \
                     decay=decay,     \
                     amsgrad=False)
    print ("Using Adam as the optimizer ...")
    print ("Optimizer parameters (recommended default): ")
    print ("\n lr={} (0.001),     \
            \n beta_1={} (0.9),   \
            \n beta_2={} (0.999), \
            \n epsilon={} (1e-08), \
            \n decay={} (0.0)".format(lr, 
                                      beta_1, 
                                      beta_2, 
                                      epsilon, 
                                      decay))
  elif optimizer_val.lower() == 'amsgrad':
    optimizer = Adam(lr=lr,           \
                     beta_1=beta_1,   \
                     beta_2=beta_2,    \
                     epsilon=epsilon, \
                     decay=decay,     \
                     amsgrad=True)
    print ("Using AmsGrad variant of Adam as the optimizer ...")
    print ("Optimizer parameters (recommended default): ")
    print ("\n lr={} (0.001),     \
            \n beta_1={} (0.9),   \
            \n beta_2={} (0.999), \
            \n epsilon={} (1e-08), \
            \n decay={} (0.0)".format(lr, 
                                      beta_1, 
                                      beta_2, 
                                      epsilon, 
                                      decay))
  elif optimizer_val.lower() == 'adamax':  
    optimizer = Adamax(lr=lr,           \
                       beta_1=beta_1,   \
                       beta_2=beta_2,    \
                       epsilon=epsilon, \
                       decay=decay)
    print ("Using Adamax variant of Adam as the optimizer ...")
    print ("Optimizer parameters (recommended default): ")
    print ("\n lr={} (0.002),     \
            \n beta_1={} (0.9),   \
            \n beta_2={} (0.999), \
            \n epsilon={} (1e-08), \
            \n schedule_decay={} (0.0)".format(lr, 
                                               beta_1, 
                                               beta_2, 
                                               epsilon, 
                                               decay))
  elif optimizer_val.lower() == 'nadam':  
    optimizer = Nadam(lr=lr,            \
                      beta_1=beta_1,    \
                      beta_2=beta_2,     \
                      epsilon=epsilon,  \
                      schedule_decay=decay)
    print ("Using Nesterov Adam optimizer ...\
           \n decay arguments is passed on to schedule_decay variable ...")
    print ("Optimizer parameters (recommended default): ")
    print ("\n lr={} (0.002),     \
            \n beta_1={} (0.9),   \
            \n beta_2={} (0.999), \
            \n epsilon={} (1e-08), \
            \n schedule_decay={} (0.004)".format(lr, 
                                                 beta_1, 
                                                 beta_2, 
                                                 epsilon, 
                                                 decay))
  else:
      optimizer = DEFAULT_OPTIMIZER
      print ("Using stochastic gradient descent with Nesterov momentum ('nsgd') as the default optimizer ...")
      print ("Options for optimizer are: 'sgd',        \
                                         \n'nsgd',     \
                                         \n'rmsprop',  \
                                         \n'adagrad',  \
                                         \n'adadelta', \
                                         \n'adam',     \
                                         \n'nadam',    \
                                         \n'amsgrad',  \
                                         \n'adamax' ...")
  return optimizer

In [0]:
def process_model(args, 
                  model, 
                  base_model, 
                  optimizer, 
                  loss, 
                  checkpointer_savepath):
  load_weights_ = args.load_weights[0]
  fine_tune_model = args.fine_tune[0]
  load_checkpoint = args.load_checkpoint[0]
   
  if load_weights_ == True:     
      try:
          with open(args.config_file[0]) as json_file:
              model_json = json_file.read()
          model = model_from_json(model_json)
      except:
          model = model
      try:
          model.load_weights(args.weights_file[0])
          print ("\nLoaded model weights from: " + str(args.weights_file[0]))
      except:
          print ("\nError loading model weights ...")
          print ("Tabula rasa ...")
          print ("Loaded default model weights ...")
  elif load_checkpoint == True and os.path.exists(checkpointer_savepath):     
      try:
          model = load_model(checkpointer_savepath)
          print ("\nLoaded model from checkpoint: " + str(checkpointer_savepath))
      except:
          if os.path.exists(args.saved_chkpnt[0]):
            model = load_model(args.saved_chkpnt[0])
            print ('\nLoaded saved checkpoint file ...')
          else:
            print ("\nError loading model checkpoint ...")
            print ("Tabula rasa ...")
            print ("Loaded default model weights ...")
  else:
      model = model
      print ("\nTabula rasa ...")
      print ("Loaded default model weights ...")
 
  try:
      NB_FROZEN_LAYERS = args.frozen_layers[0]
  except:
      NB_FROZEN_LAYERS = DEFAULT_NB_LAYERS_TO_FREEZE
      
  if fine_tune_model == True:
      print ("\nFine tuning Inception architecture ...")
      print ("Frozen layers: " + str(NB_FROZEN_LAYERS))
      model = finetune_model(model, optimizer, loss, NB_FROZEN_LAYERS)
  else:
      print ("\nTransfer learning using Inception architecture ...")
      model = transferlearn_model(model, base_model, optimizer, loss)
      
  return model

In [0]:
def process_images(args):  
  train_aug = args.train_aug[0] 
  test_aug = args.test_aug[0] 
   
  if str((args.base_model[0]).lower()) == 'inceptionv4' or  \
     str((args.base_model[0]).lower()) == 'inception_v4' or \
     str((args.base_model[0]).lower()) == 'inception_resnet':
      preprocess_input = preprocess_input_inceptionv4
  else:
      preprocess_input = preprocess_input_inceptionv3
  
  if train_aug==True:
    try:
        train_rotation_range = args.train_rot[0]
        train_width_shift_range = args.train_w_shift[0]
        train_height_shift_range = args.train_ht_shift[0]
        train_shear_range = args.train_shear[0]
        train_zoom_range = args.train_zoom[0]
        train_vertical_flip = args.train_vflip[0]
        train_horizontal_flip = args.train_hflip[0]
    except:
        train_rotation_range = 30
        train_width_shift_range = 0.2
        train_height_shift_range = 0.2
        train_shear_range = 0.2
        train_zoom_range = 0.2
        train_vertical_flip = True
        train_horizontal_flip = True
        print ("\nFailed to load custom training image augmentation parameters ...")
        print ("Loaded pre-set defaults ...")
        print ("To switch off image augmentation during training, set --train_augmentation flag to False")
        
    train_datagen =  ImageDataGenerator(preprocessing_function=preprocess_input,
                                        rotation_range=train_rotation_range,
                                        width_shift_range=train_width_shift_range,
                                        height_shift_range=train_height_shift_range,
                                        shear_range=train_shear_range,
                                        zoom_range=train_zoom_range,
                                        vertical_flip=train_vertical_flip,                                  
                                        horizontal_flip=train_horizontal_flip)
    print ("\nCreated image augmentation pipeline for training images ...")     
    print ("Image augmentation parameters for training images: \
          \n image rotation range = {},\
          \n width shift range = {},\
          \n height shift range = {}, \
          \n shear range = {} ,\
          \n zoom range = {}, \
          \n enable vertical flip = {}, \
          \n enable horizontal flip = {}".format(train_rotation_range,
                                                   train_width_shift_range,
                                                   train_height_shift_range,
                                                   train_shear_range,
                                                   train_zoom_range,
                                                   train_vertical_flip,
                                                   train_horizontal_flip))
  else:
      train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
  
  if test_aug==True:
      try:
        test_rotation_range = args.test_rot[0]
        test_width_shift_range = args.test_w_shift[0]
        test_height_shift_range = args.test_ht_shift[0]
        test_shear_range = args.test_shear[0]
        test_zoom_range = args.test_zoom[0]
        test_vertical_flip = args.test_vflip[0]
        test_horizontal_flip = args.test_hflip[0]
      except:
        test_rotation_range = 30
        test_width_shift_range = 0.2
        test_height_shift_range = 0.2
        test_shear_range = 0.2
        test_zoom_range = 0.2
        test_vertical_flip = True
        test_horizontal_flip = True
        print ("\nFailed to load custom training image augmentation parameters ...")
        print ("Loaded pre-set defaults ...")
        print ("To switch off image augmentation during training, set --train_augmentation flag to False")
      test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                        rotation_range=test_rotation_range,
                                        width_shift_range=test_width_shift_range,
                                        height_shift_range=test_height_shift_range,
                                        shear_range=test_shear_range,
                                        zoom_range=test_zoom_range,
                                        vertical_flip=test_vertical_flip,
                                        horizontal_flip=test_horizontal_flip)
      print ("\nCreated image augmentation pipeline for training images ...")     
      print ("\nImage augmentation parameters for training images:")
      print( "\n image rotation range = {},\
              \n width shift range = {},\
              \n height shift range = {}, \
              \n shear range = {} ,\
              \n zoom range = {}, \
              \n enable vertical flip = {}, \
              \n enable horizontal flip = {}".format(test_rotation_range,
                                                     test_width_shift_range,
                                                     test_height_shift_range,
                                                     test_shear_range,
                                                     test_zoom_range,
                                                     test_vertical_flip,
                                                     test_horizontal_flip))
  else:
      test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

  return [train_datagen, test_datagen]

In [0]:
def gen_model(args, enable_dropout):
  if str((args.base_model[0]).lower()) == 'inceptionv4' or  \
     str((args.base_model[0]).lower()) == 'inception_v4' or \
     str((args.base_model[0]).lower()) == 'inception_resnet':
      base_model = InceptionResNetV2(weights='imagenet', \
                                     include_top=False)
      base_model_name = 'Inception version 4'
  else:
      base_model = InceptionV3(weights='imagenet', 
                               include_top=False)
      base_model_name = 'Inception version 3'
  print ('\nBase model: ' + str(base_model_name))
  nb_classes = len(glob.glob(args.train_dir[0] + "/*"))
  model = add_top_layer(args, 
                        enable_dropout,
                        base_model, 
                        nb_classes)
  print ("New top layer added to: " + str(base_model_name))
  return [model, base_model]

In [0]:
def train(args): 
  """
    A function that takes the user arguments and initiates a training session of the neural network.
    
    This function takes only one input: args
    
    Example usage:
            
        if train_model == True:
            print ("Training sesssion initiated ...")
            train(args)
  """    
  
  if not os.path.exists(args.output_dir[0]):
    os.makedirs(args.output_dir[0])
    
  optimizer  = select_optimizer(args)
  loss = args.loss[0]
  checkpointer_savepath = os.path.join(args.output_dir[0]     +       
                                       '/checkpoint/Transfer_learn_' +       
                                       str(IM_WIDTH)  + '_'  + 
                                       str(IM_HEIGHT) + '_'  + '.h5')
  
  nb_train_samples = get_nb_files(args.train_dir[0])
  nb_classes = len(glob.glob(args.train_dir[0] + "/*"))
  
  print ("\nTotal number of training samples = " + str(nb_train_samples))
  print ("Number of training classes = " + str(nb_classes))
  
  nb_val_samples = get_nb_files(args.val_dir[0])
  nb_val_classes = len(glob.glob(args.val_dir[0] + "/*"))
  
  print ("\nTotal number of validation samples = " + str(nb_val_samples))
  print ("Number of validation classes = " + str(nb_val_classes))
  
  if nb_val_classes == nb_classes:
      print ("\nInitiating training session ...")
  else:
      print ("\nMismatched number of training and validation data classes ...")
      print ("Unequal number of sub-folders found between train and validation directories ...")
      print ("Each sub-folder in train and validation directroies are treated as a separate class ...")
      print ("Correct this mismatch and re-run ...")
      print ("\nNow exiting ...")
      sys.exit(1)
      
  nb_epoch = int(args.epoch[0])
  batch_size = int(args.batch[0])    
  
  [train_datagen, validation_datagen] = process_images(args)
  
  labels = generate_labels(args)
  
  train_dir = args.train_dir[0]
  val_dir = args.val_dir[0]
  
  if args.normalize[0] and os.path.exists(args.root_dir[0]):
      normalize(args, 
                labels, 
                move = False,
                sub_sample = args.sub_sample[0])
      train_dir = os.path.join(args.root_dir[0] + 
                               str ('/.tmp_train/'))
      val_dir = os.path.join(args.root_dir[0] + 
                             str ('/.tmp_validation/'))
      
  print ("\nGenerating training data: ... ")
  train_generator = train_datagen.flow_from_directory(train_dir,
                                                      target_size=(IM_WIDTH, IM_HEIGHT),
                                                      batch_size=batch_size,
                                                      class_mode='categorical')
  
  print ("\nGenerating validation data: ... ")
  validation_generator = validation_datagen.flow_from_directory(val_dir,
                                                          target_size=(IM_WIDTH, IM_HEIGHT),
                                                          batch_size=batch_size,
                                                          class_mode='categorical')
  
  
  [model, base_model] = gen_model(args, enable_dropout)
    
  model = process_model(args, 
                        model, 
                        base_model, 
                        optimizer, 
                        loss, 
                        checkpointer_savepath)
            
  print ("\nInitializing training with  class labels: " + 
         str(labels))
  
  model_summary_ = args.model_summary[0]
  
  if model_summary_ == True:
      print (model.summary())
  else:
      print ("\nSuccessfully loaded deep neural network classifier for training ...")
      print ("\nReady, Steady, Go ...")
      print ("\n")
        
  if not os.path.exists(os.path.join(args.output_dir[0] + '/checkpoint/')):
    os.makedirs(os.path.join(args.output_dir[0] + '/checkpoint/'))
    
  lr = args.learning_rate[0]
    
  earlystopper = EarlyStopping(patience=6, 
                               verbose=1)
  checkpointer = ModelCheckpoint(checkpointer_savepath, 
                                 verbose=1,  
                                 save_best_only=True)
  learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                              patience=2,
                                              mode = 'max',
                                              epsilon=1e-4, 
                                              cooldown=1,
                                              verbose=1, 
                                              factor=0.5, 
                                              min_lr=lr*1e-2)
  
  model_train = model.fit_generator(train_generator,
                                    epochs=nb_epoch,
                                    steps_per_epoch=nb_train_samples//20,
                                    validation_data=validation_generator,
                                    validation_steps=nb_val_samples//20,
                                    class_weight='auto', 
                                    callbacks=[earlystopper, 
                                               learning_rate_reduction, 
                                               checkpointer])
  
  if args.fine_tune[0] == True:
      save_model(args, "_ft_", model)
      generate_plot(args, "_ft_", model_train)
  else:
      save_model(args, "_tl_", model)
      generate_plot(args, "_tl_", model_train)

In [0]:
NB_FROZEN_LAYERS=170
enable_training=True
enable_dropout=True

In [0]:
import types
args=types.SimpleNamespace()
args.base_model=['Inception_V4']
args.frozen_layers=[NB_FROZEN_LAYERS]
args.optimizer_val=['amsgrad']
args.decay=[0.0]
args.beta_2=[0.999]
args.beta_1=[0.9]
args.rho=[0.9]
args.learning_rate=[1e-7]
args.loss=['categorical_crossentropy']
args.activation=['relu']
args.epsilon=[1e-8]
args.dropout=[0.4]
args.test_hflip=[True]
args.test_vflip=[True]
args.test_zoom=[True]
args.test_shear=[True]
args.train_model=[enable_training]
args.output_dir=['./{}/'.format(dataset_id)]
args.root_dir=['./']
args.val_dir=['./{}/validation/'.format(dataset_id)]
args.train_dir=['./{}/train/'.format(dataset_id)]
args.epoch=[20]
args.batch=[10]
args.train_aug=[True]
args.test_aug=[True]
args.normalize=[False]
args.sub_sample=[False]
args.load_weights=[False]
args.fine_tune=[True]
args.load_checkpoint=[True]
args.model_summary=[False]
args.plot=[True]

In [0]:
import os 
import sys
import subprocess
import gc

In [0]:
def execute_in_shell(command=None, 
                     verbose = False):
    """ 
        command -- keyword argument, takes a list as input
        verbsoe -- keyword argument, takes a boolean value as input
    
        This is a function that executes shell scripts from within python.
        
        Keyword argument 'command', should be a list of shell commands.
        Keyword argument 'verbose', should be a boolean value to set verbose level.
        
        Example usage: execute_in_shell(command = ['ls ./some/folder/',
                                                    ls ./some/folder/  -1 | wc -l'],
                                        verbose = True ) 
                                        
        This command returns dictionary with elements: Output and Error.
        
        Output records the console output,
        Error records the console error messages.
                                        
    """
    error = []
    output = []
    
    if isinstance(command, list):
        for i in range(len(command)):
            try:
                process = subprocess.Popen(command[i], shell=True, stdout=subprocess.PIPE)
                process.wait()
                out, err = process.communicate()
                error.append(err)
                output.append(out)
                if verbose:
                    print ('Success running shell command: {}'.format(command[i]))
            except Exception as e:
                print ('Failed running shell command: {}'.format(command[i]))
                if verbose:
                    print(type(e))
                    print(e.args)
                    print(e)
                
    else:
        print ('The argument command takes a list input ...')
    return {'Output': output, 'Error': error }

In [57]:
if setup:
  execute_in_shell(command = command, 
                   verbose = True)

Success running shell command: pip3 install -q kaggle PyDrive scikit-optimize >/dev/null 2>&1
Success running shell command: mkdir /content/
Success running shell command: mkdir /content/.kaggle/
Success running shell command: mkdir ./kaggle_diabetic_retinopathy/


In [58]:
IM_WIDTH, IM_HEIGHT = 299, 299                                                  # Default input image size for Inception v3 and v4 architecture
DEFAULT_EPOCHS = 100
DEFAULT_BATCHES = 20
FC_SIZE = 4096
DEFAULT_DROPOUT = 0.1
DEFAULT_NB_LAYERS_TO_FREEZE = 169

verbose = False

sgd = SGD(lr=1e-7, decay=0.5, momentum=1, nesterov=True)
rms = RMSprop(lr=1e-7, rho=0.9, epsilon=1e-08, decay=0.0)
ada = Adagrad(lr=1e-3, epsilon=1e-08, decay=0.0)

Instructions for updating:
Colocations handled automatically by placer.


In [65]:
command = ['rm -r ./{}/train/train/'.format(dataset_id)]
execute_in_shell(command=command)

{'Error': [None], 'Output': [b'']}

In [0]:
if enable_training:
  train(args)

Using AmsGrad variant of Adam as the optimizer ...
Optimizer parameters (recommended default): 

 lr=1e-07 (0.001),                 
 beta_1=0.9 (0.9),               
 beta_2=0.999 (0.999),             
 epsilon=1e-08 (1e-08),             
 decay=0.0 (0.0)

Total number of training samples = 34126
Number of training classes = 5

Total number of validation samples = 1000
Number of validation classes = 5

Initiating training session ...

Failed to load custom training image augmentation parameters ...
Loaded pre-set defaults ...
To switch off image augmentation during training, set --train_augmentation flag to False

Created image augmentation pipeline for training images ...
Image augmentation parameters for training images:           
 image rotation range = 30,          
 width shift range = 0.2,          
 height shift range = 0.2,           
 shear range = 0.2 ,          
 zoom range = 0.2,           
 enable vertical flip = True,           
 enable horizontal flip = True

Failed to